In [6]:
# Importar las librerías necesarias
import pandas as pd

# Cargar los datos de un archivo CSV

df_superstore = pd.read_csv('C:/Users/ljgov/Documents/LABORATORIA/PROYECTO_ETL/superstore.csv')


In [10]:
import pandas as pd

# Habilitar la opción para evitar advertencias en el futuro
pd.set_option('future.no_silent_downcasting', True)

# Ejemplo de datos iniciales
data = {
    'category': ['Furniture', 'Office Supplies', 'Technology', 'furniture'],
    'city': ['Los Angeles', 'Los Angeles', 'los angeles', 'Los Angeles'],
    'country': ['United States', 'US', 'U.S.', 'United States'],
    'customer_ID': ['LS-172304', 'MV-174584', 'CS-123104', 'AP-109154'],
    'order_priority': ['High', 'Medium', 'Low', 'medium'],
    'market': ['US', 'US', 'US', 'US'],
    'order_date': ['7/01/2011 0:00:00', '8/01/2011 0:00:00', '9/01/2011 0:00:00', '10/01/2011 0:00:00'],
    'region': ['West', 'West', 'west', 'West'],
    'ship_date': ['9/01/2011 0:00:00', '10/01/2011 0:00:00', '11/01/2011 0:00:00', '12/01/2011 0:00:00'],
    'product_name': ['Desk', 'Pen', 'Laptop', 'Chair'],
    'profit': [50, 5, 150, 45],
    'quantity': [2, 10, 1, 3],
    'sales': [200, 20, 1000, 150],
    'ship_mode': ['Standard', 'Second Class', 'First Class', 'Standard'],
    'state': ['California', 'California', 'California', 'California'],
    'sub_category': ['Office Supplies', 'Office Supplies', 'Technology', 'Furniture'],
    'year': [2011, 2011, 2011, 2011],
    'market2': ['West', 'West', 'West', 'West'],
    'weeknum': [1, 2, 3, 4],
    'order_id': ['OD001', 'OD002', 'OD003', 'OD004'],
    'row_id': [1, 2, 3, 4],
    'customer_name': ['John Doe', 'Jane Smith', 'Alan Smithee', 'Emma Stone'],
    'discount': [0.1, 0.15, 0.2, 0.05],
    'shipping_cost': [15, 7, 20, 10],
    'unknown': ['NA', 'NA', 'NA', 'NA']
}

df = pd.DataFrame(data)

# Cambiar el valor de la columna 'unknown' de 'NA' a 1 y convertir a entero
df['unknown'] = df['unknown'].replace('NA', 1).astype(int)

# 1. Guardar las columnas originales
df['order_priority_original'] = df['order_priority']

# 2. Estandarización de textos: Solo la primera letra mayúscula
df['category'] = df['category'].str.capitalize()
df['city'] = df['city'].str.capitalize()
df['country'] = df['country'].str.capitalize()
df['order_priority'] = df['order_priority'].str.capitalize()
df['region'] = df['region'].str.capitalize()

# 3. Corrección de errores tipográficos y normalización
df['country'] = df['country'].replace(['Us', 'U.s.'], 'United states')
df['order_priority'] = df['order_priority'].replace({
    'Critical': 4,
    'High': 3,
    'Medium': 2,
    'Low': 1
}).astype(int)

# 4. Codificación de variables categóricas
category_mapping = {
    'Furniture': 1,
    'Office supplies': 2,
    'Technology': 3
}

df['category_code'] = df['category'].map(category_mapping)
df['category_code'] = df['category_code'].fillna(-1)

# 5. Conversión de las columnas de fecha al formato de fecha estándar
df['order_date'] = pd.to_datetime(df['order_date'], format='%d/%m/%Y %H:%M:%S').dt.date
df['ship_date'] = pd.to_datetime(df['ship_date'], format='%d/%m/%Y %H:%M:%S').dt.date

# 6. Extraer componentes de la fecha
df['order_year'] = pd.DatetimeIndex(df['order_date']).year
df['order_month'] = pd.DatetimeIndex(df['order_date']).month
df['order_day'] = pd.DatetimeIndex(df['order_date']).day

df['ship_year'] = pd.DatetimeIndex(df['ship_date']).year
df['ship_month'] = pd.DatetimeIndex(df['ship_date']).month
df['ship_day'] = pd.DatetimeIndex(df['ship_date']).day

# 7. Reordenar las columnas
df = df[['category', 'category_code', 'city', 'country', 'customer_ID', 'customer_name', 'discount',
         'market', 'unknown', 'order_date', 'order_year', 'order_month', 'order_day', 'order_id',
         'order_priority_original', 'order_priority', 'product_name', 'profit',
         'quantity', 'region', 'row_id', 'sales', 'ship_date', 'ship_year', 
         'ship_month', 'ship_day', 'ship_mode', 'shipping_cost', 'state', 'sub_category',
         'year', 'market2', 'weeknum']]

# 8. Verificación de resultados
print(df)


          category  category_code         city        country customer_ID  \
0        Furniture              1  Los angeles  United states   LS-172304   
1  Office supplies              2  Los angeles  United states   MV-174584   
2       Technology              3  Los angeles  United states   CS-123104   
3        Furniture              1  Los angeles  United states   AP-109154   

  customer_name  discount market  unknown  order_date  ...  ship_year  \
0      John Doe      0.10     US        1  2011-01-07  ...       2011   
1    Jane Smith      0.15     US        1  2011-01-08  ...       2011   
2  Alan Smithee      0.20     US        1  2011-01-09  ...       2011   
3    Emma Stone      0.05     US        1  2011-01-10  ...       2011   

   ship_month  ship_day     ship_mode shipping_cost       state  \
0           1         9      Standard            15  California   
1           1        10  Second Class             7  California   
2           1        11   First Class          